In [ ]:
#@title #Install library  { display-mode: "form" }
!pip install gender-guesser


In [ ]:
#@title #1. Library { display-mode: "form" }

# importar librerias
import os
import re
import gc
import warnings
import pandas as pd
import gspread as gs
from google.colab import files, drive, auth, data_table
from google.auth import default
import gender_guesser.detector as gender

In [ ]:
#@title ##Gender inference
# Data String
df_m['NOMBRES'] = df_m['NOMBRES'].astype(str)
d = gender.Detector(case_sensitive=False)

# Gender estimation
df_m = df_m.assign(Longitud_nombres = df_m['NOMBRES'].apply(lambda x: len(x.split())))
df_m = df_m.assign(NombreCliente_ = df_m['NOMBRES'].apply(lambda x: d.get_gender(x.split()[0]).split('_')[-1]))
df_m['NombreCliente_1'] = 0
df_m['NombreCliente_2'] = 0
df_m['NombreCliente_1'][df_m.Longitud_nombres==2] = df_m['NOMBRES'][df_m.Longitud_nombres==2].apply(lambda x: d.get_gender(x.split()[0]).split('_')[-1])
df_m['NombreCliente_2'][df_m.Longitud_nombres==2] = df_m['NOMBRES'][df_m.Longitud_nombres==2].apply(lambda x: d.get_gender(x.split()[1]).split('_')[-1])

# Data information by gender
ym = ( (df_m['NombreCliente_'] == 'male') ).astype(int)
yf = ( (df_m['NombreCliente_'] == 'female') | (df_m['NombreCliente_'] == 'andy') ).astype(int)
yu = ( (df_m['NombreCliente_'] == 'unknown')).astype(int)

ym1 = ( (df_m['NombreCliente_1'] == 'male') ).astype(int)
yf1 = ( (df_m['NombreCliente_1'] == 'female') | (df_m['NombreCliente_1'] == 'andy') ).astype(int)
yu1 = ( (df_m['NombreCliente_1'] == 'unknown')).astype(int)

ym2 = ( (df_m['NombreCliente_2'] == 'male') ).astype(int)
yf2 = ( (df_m['NombreCliente_2'] == 'female') | (df_m['NombreCliente_2'] == 'andy') ).astype(int)
yu2 = ( (df_m['NombreCliente_2'] == 'unknown')).astype(int)

# Inficador de cada columna por género
df_m = df_m.assign(Male=ym)
df_m = df_m.assign(Female=yf)
df_m = df_m.assign(Unknown=yu)

df_m = df_m.assign(Male1=ym)
df_m = df_m.assign(Female1=yf)
df_m = df_m.assign(Unknown1=yu)

df_m = df_m.assign(Male2=ym)
df_m = df_m.assign(Female2=yf)
df_m = df_m.assign(Unknown2=yu)

indx = df_m[df_m['NombreCliente_']=='unknown'].index.tolist()
for i in indx:
  if (df_m.Male1.iloc[i]==1) and (df_m.Male2.iloc[i]==1):
    df_m.Male.iloc[i] = 1
    df_m.Unknown.iloc[i] = 0
    df_m.NombreCliente_.iloc[i] = 'male'
  elif (df_m.Male1.iloc[i]==1) | (df_m.Male2.iloc[i]==0):
    df_m.Male.iloc[i] = 1
    df_m.Unknown.iloc[i] = 0
    df_m.NombreCliente_.iloc[i] = 'male'
  elif (df_m.Male1.iloc[i]==0) | (df_m.Male2.iloc[i]==1):
    df_m.Male.iloc[i] = 1
    df_m.Unknown.iloc[i] = 0
    df_m.NombreCliente_.iloc[i] = 'male'
  elif (df_m.Female1.iloc[i]==1) and (df_m.Female2.iloc[i]==1):
    df_m.Female.iloc[i] = 1
    df_m.Unknown.iloc[i] = 0
    df_m.NombreCliente_.iloc[i] = 'female'
  elif (df_m.Female1.iloc[i]==1) | (df_m.Female2.iloc[i]==0):
    df_m.Female.iloc[i] = 1
    df_m.Unknown.iloc[i] = 0
    df_m.NombreCliente_.iloc[i] = 'female'
  elif (df_m.Female1.iloc[i]==0) | (df_m.Female2.iloc[i]==1):
    df_m.Female.iloc[i] = 1
    df_m.Unknown.iloc[i] = 0
    df_m.NombreCliente_.iloc[i] = 'female'

df_m.drop(columns=['NombreCliente_', 'NombreCliente_1', 'NombreCliente_2',
                   'Male1', 'Female1', 'Unknown1','Male2',	'Female2', 'Unknown2'],
          axis=1, inplace=True)

df_m.head(5)